## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [16]:
from keras.datasets import cifar10
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import tensorflow as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.compat.v1.ConfigProto(allow_soft_placement=True)

#最多占gpu资源的70%
gpu_options =tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.7)

#开始不会给tensorflow全部gpu资源 而是按需增加
# config.gpu_options.allow_growth = True
# sess = tf.Session(config=config)

In [17]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [18]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, 32, 32, 16)   448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_28 (BatchNo (None, 32, 32, 16)   64          conv2d_31[0][0]                  
__________________________________________________________________________________________________
activation_28 (Activation)      (None, 32, 32, 16)   0           batch_normalization_28[0][0]     
_______________________________________________________________________________________

In [19]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/30
782/782 [==============================] - 3237s 4s/step - loss: 1.8057 - accuracy: 0.5208 - val_loss: 1.6040 - val_accuracy: 0.5719
Epoch 2/30
782/782 [==============================] - 3255s 4s/step - loss: 1.3163 - accuracy: 0.6661 - val_loss: 1.5641 - val_accuracy: 0.5959
Epoch 3/30
782/782 [==============================] - 3319s 4s/step - loss: 1.1172 - accuracy: 0.7226 - val_loss: 1.4126 - val_accuracy: 0.6305
Epoch 4/30
782/782 [==============================] - 3301s 4s/step - loss: 0.9885 - accuracy: 0.7646 - val_loss: 1.3486 - val_accuracy: 0.6469
Epoch 5/30
782/782 [==============================] - 3272s 4s/step - loss: 0.8918 - accuracy: 0.7942 - val_loss: 1.2406 - val_accuracy: 0.6872
Epoch 6/30
782/782 [==============================] - 3285s 4s/step - loss: 0.8206 - accuracy: 0.8191 - val_loss: 1.4171 - val_accuracy: 0.6562
Epoch 7/30
782/782 [==============================] - 3281s 4s/step - loss: 0.7651 - accuracy: 0.8376 - val_loss: 1.1763 - val_accuracy: